In [ ]:
import folium
import pyproj
import momepy
import shapely
import osmnx as ox
import pandas as pd
import networkx as nx
import geopandas as gpd
import matplotlib.pyplot as plt

from tqdm import tqdm
from shapely.wkt import loads, dumps
from shapely.ops import nearest_points
from shapely.geometry import Point, Polygon, LineString, MultiLineString

In [ ]:
maxspeed = {
 'motorway':        110, # Автомагистрали 
 'motorway_link':   110, # Съезды на развязках дорог, на которых действуют те же правила движения, что и на (motorway).
 'primary':         80,  # Автомобильные дороги регионального значения
 'primary_link':    80,  # Съезды на развязках дорог с той же важностью в дорожной сети, что и primary.
 'residential':     60,  # Дороги, которые проходят внутри жилых зон, а также используются для подъезда к ним. 
 'secondary':       70,  # Автомобильные дороги областного значения
 'secondary_link':  70,  # Съезды на развязках дорог с той же важностью в дорожной сети, что и secondary.
 'tertiary':        60,  # Более важные автомобильные дороги среди прочих 
                         # автомобильных дорог местного значения, например
                         # соединяющие районные центры с сёлами, а также несколько сёл между собой.
 'tertiary_link':   60,  # Съезды на развязках дорог с той же важностью в дорожной сети, что и tertiary.
 'trunk':           90,  # Важные дороги, не являющиеся автомагистралями
 'trunk_link':      90,  # Съезды на развязках дорог с той же важностью в дорожной сети, что и trunk.
 'unclassified':    60   # Остальные автомобильные дороги местного значения, образующие соединительную сеть дорог.
 }

In [ ]:
spb = ox.geocode_to_gdf('R337422', by_osmid=True) # Санкт-Петербург
lo = ox.geocode_to_gdf('R176095', by_osmid=True)  # Ленинградская область

spb_buff = spb.to_crs(epsg=3857).buffer(1000).to_crs(epsg=4326)
lo_buff = lo.to_crs(epsg=3857).buffer(3000).to_crs(epsg=4326)

result_polygon = lo_buff.difference(spb.unary_union)

lo_filter = {'highway': ['motorway', 'trunk', 'primary', 'secondary', 
                           'tertiary', 'unclassified', 'residential', 
                           'motorway_link', 'trunk_link', 'primary_link', 'secondary_link', 'tertiary_link']}
spb_filter = {'highway': ['motorway', 'trunk', 'primary', 
                           'motorway_link', 'trunk_link', 'primary_link']}

polygon_boundary = result_polygon.unary_union
polygon_boundary_spb = spb_buff.unary_union

lo_gdf = ox.features_from_polygon(polygon_boundary,      tags=lo_filter)
spb_gdf = ox.features_from_polygon(polygon_boundary_spb, tags=spb_filter)

lo_gdf = lo_gdf[lo_gdf['geometry'].geom_type == 'LineString']

roads_within_polygon = gpd.sjoin(lo_gdf, lo, how='inner', predicate='intersects')
roads_within_polygon.reset_index(inplace=True)

polygon = lo.union(spb_buff)
gdf_polygon = gpd.GeoDataFrame(geometry=polygon)

roads_within_polygon['exit'] = 0
roads_within_polygon['length'] = roads_within_polygon.to_crs(epsg=3857).geometry.length
roads_within_polygon['maxspeed'] = roads_within_polygon['highway'].map(maxspeed)
roads_within_polygon['time'] = roads_within_polygon['length'] / roads_within_polygon['maxspeed']

In [ ]:
intersections = roads_within_polygon['geometry'].intersection(gdf_polygon.unary_union)
roads_within_polygon['geometry'] = intersections
roads_within_polygon = roads_within_polygon[~roads_within_polygon['geometry'].is_empty]

mask = roads_within_polygon.intersects(gdf_polygon.unary_union.boundary)
roads_within_polygon.loc[mask, 'exit'] = 1

In [ ]:
roads_within_polygon_exploded = roads_within_polygon.explode().reset_index(drop=True)
graph = momepy.gdf_to_nx(roads_within_polygon_exploded.to_crs(epsg=4326))

In [ ]:
components = list(nx.connected_components(graph))
largest_component = max(components, key=len)
graph = graph.subgraph(largest_component).copy()

In [ ]:
for node in graph.nodes:
    graph.nodes()[node]['x'] = node[0]
    graph.nodes()[node]['y'] = node[1]

for node in graph.nodes:
    graph.nodes[node]['exit'] = 0

for u,v,d in graph.edges(data=True):
    graph.nodes[u]['highway'] = d['highway']
    graph.nodes[v]['highway'] = d['highway']

for node in graph.nodes:
    point = Point(graph.nodes[node]['x'], graph.nodes[node]['y'])
    if point.buffer(0.0001).intersects(gdf_polygon.unary_union.boundary):
        graph.nodes[node]['exit'] = 1

In [ ]:
import folium

mymap = folium.Map(tiles='CartoDB Dark_Matter')

# Создаем FeatureGroup для узлов
nodes_group = folium.FeatureGroup(name='Exit Nodes')
edges_group = folium.FeatureGroup(name='Roads')
other_group = folium.FeatureGroup(name='Other_roads')

nodes_ = [(u, d) for u, d in graph.nodes(data=True) if d.get('exit') == 1]
all_roads_type = ['motorway', 'motorway_link', 'primary', 'primary_link', 'secondary', 'secondary_link', 'tertiary', 'tertiary_link', 'trunk', 'trunk_link']
weights = {
 'motorway':        5, # Автомагистрали 
 'motorway_link':   5, # Съезды на развязках дорог, на которых действуют те же правила движения, что и на (motorway).
 'primary':         3,  # Автомобильные дороги регионального значения
 'primary_link':    3,  # Съезды на развязках дорог с той же важностью в дорожной сети, что и primary.
 'residential':     1,  # Дороги, которые проходят внутри жилых зон, а также используются для подъезда к ним. 
 'secondary':       2,  # Автомобильные дороги областного значения
 'secondary_link':  2,  # Съезды на развязках дорог с той же важностью в дорожной сети, что и secondary.
 'tertiary':        1,  # Более важные автомобильные дороги среди прочих 
                         # автомобильных дорог местного значения, например
                         # соединяющие районные центры с сёлами, а также несколько сёл между собой.
 'tertiary_link':   1,  # Съезды на развязках дорог с той же важностью в дорожной сети, что и tertiary.
 'trunk':           4,  # Важные дороги, не являющиеся автомагистралями
 'trunk_link':      4,  # Съезды на развязках дорог с той же важностью в дорожной сети, что и trunk.
 'unclassified':    1   # Остальные автомобильные дороги местного значения, образующие соединительную сеть дорог.
 }

# Отображение на карте
for node in nodes_:
    if graph.nodes[node[0]]['highway'] in all_roads_type:
        folium.CircleMarker(
            [graph.nodes[node[0]]['y'], graph.nodes[node[0]]['x']],
            radius=5,  # Уменьшенный размер кружка
            color="yellow",
            opacity=1,
            fill=True,  # Включаем заливку
            fill_color="yellow",  # Цвет заливки
            tooltip=f"highway: {graph.nodes[node[0]]['highway']}"
        ).add_to(nodes_group)
nodes_group.add_to(mymap)

for u, v, d in graph.edges(data=True):
    if d.get('highway') in all_roads_type:
        folium.PolyLine(
            [[graph.nodes[u]['y'], graph.nodes[u]['x']],
             [graph.nodes[v]['y'], graph.nodes[v]['x']]],
            color="yellow",
            weight=weights[d.get('highway')],  # Толщина линии
            opacity=1,
            tooltip=f"highway: {d.get('highway')}"
        ).add_to(edges_group)
edges_group.add_to(mymap)

for u, v, d in graph.edges(data=True):
    if d.get('highway') not in all_roads_type:
        folium.PolyLine(
            [[graph.nodes[u]['y'], graph.nodes[u]['x']],
             [graph.nodes[v]['y'], graph.nodes[v]['x']]],
            color="yellow",
            weight=0.5,  # Толщина линии
            opacity=1,
            tooltip=f"highway: {d.get('highway')}"
        ).add_to(other_group)
other_group.add_to(mymap)

folium.GeoJson(
    gdf_polygon.unary_union.boundary,
    name='OSM',
    style_function=lambda feature: {
        'color': 'green',        # Цвет границы
        'weight': 1.5,           # Толщина границы
    }
).add_to(mymap)

# Добавляем контроллер слоев
folium.LayerControl().add_to(mymap)

# Сохраняем карту в HTML файл
mymap.save('data/html/roads_exit.html')